In [1]:
#from games_setup import *
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import os
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
NULL_STR = ""
ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")
STATPHOSPHORYLATION = "statPhosphorylation"
PSTATDIMERISATION = "PstatDimerisation"
PSTATDIMERISATIONNUC = "PstatDimerisationNuc"
SPECIES_TEST = "species_test"
PSTAT_SOL = "Pstat_sol"
PSTATDIMER_NUC = "PstatDimer_nuc"
PSTAT_NUC = "Pstat_nuc"
STAT_SOL = "stat_sol"
PSTATDIMER_SOL = "PstatDimer_sol"

In [2]:
PREFIX = "BIOMD0000000"

def load_file_from_curated_data(num):

  format_num = format(num, '03d')
  filename = "%s%s_url.xml" % (PREFIX, format_num)
  path = os.path.join(cn.BIOMODELS_DIR, filename)
  simple = SimpleSBML()
  simple.initialize(path)
  return simple

In [3]:
simple = load_file_from_curated_data(167)
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [statPhosphorylation:  -> {species_test}]
Type III error:  [PstatDimerisation: 2.00 {Pstat_sol} -> 2.00 {Pstat_nuc=stat_nuc=stat_sol}]


In [4]:
report, error_num = gr.reportEchelonError(m.echelon_errors, explain_details=True)
print(report[-288:])

will result in empty reactant with zero mass:

:  -> {species_test}


----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************




In [5]:
extended_report = NULL_STR
extended_report = extended_report + "will result in empty reactant with zero mass:\n\n:  -> {species_test}\n"
extended_report = extended_report + "\n----------------------------------------------------------------------\n\n"
extended_report = extended_report + "\n----------------------------------------------------------------------\n\n"
extended_report = extended_report + "\n\n**********************************************************************\n\n"
print(report[-288:] == extended_report)

False


In [6]:
print(report[-288:])
print(extended_report)

will result in empty reactant with zero mass:

:  -> {species_test}


----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************


will result in empty reactant with zero mass:

:  -> {species_test}

----------------------------------------------------------------------

----------------------------------------------------------------------



**********************************************************************




In [5]:
som = m.getNode(m.simple.getMolecule(PSTATDIMER_NUC))
report, error_num = gr.reportReactionsInSOM(som, 0)
common_part = "\n1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n"
case1 = "\n{PstatDimer_sol=PstatDimer_nuc}"
case2 = "\n{PstatDimer_nuc=PstatDimer_sol}"
print(error_num == 1)
#print(report == '\n{PstatDimer_sol=PstatDimer_nuc} is inferred by:\n1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n')
print(report == (case1 + common_part) or report == (case2 + common_part))

True
False


In [6]:
report

'1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n'

In [7]:
common_part

'\n1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n'

In [16]:
case1 + common_part

'\n{PstatDimer_sol=PstatDimer_nuc}is inferred by:\n1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n'

In [18]:
report[:-338]

''

In [19]:
report

'\n{PstatDimer_nuc=PstatDimer_sol} is inferred by:\n1. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc\n'